# 0) chargement de spark

In [58]:
from pyspark.sql import SparkSession

#sans se connecter au cluster
#spark = SparkSession.builder.getOrCreate()

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

# 1) lire de la donnée

## 1.1) lecture brute
Chargez le fichier ville_1.csv dans une variable nommée df.

Vous pouvez afficher votre donnée en utilisant la méthode take() ou la methode collect() de l'objet pyspark DataFrame (attention appeler collect() sur un dataframe est déconseillé si vous avez du vrai big data).

L'objet possède aussi un attribut appelé dtypes, appelez cet attribut pour obtenir la liste des colonnes et leur type.

In [98]:
path = "hdfs://namenode:9000/Bigdata/covid.csv"
test= spark.read.format('csv').options(header= True , sep=";").load(path)
test.show()



+---+----------+-------+--------------------+------+--------+----+---+----+
|dep|      date|lib_dep|             lib_reg|tx_pos|tx_incid|hosp|rea| pos|
+---+----------+-------+--------------------+------+--------+----+---+----+
|  1|18/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   2|  0|null|
|  1|19/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   2|  0|null|
|  1|20/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   2|  0|null|
|  1|21/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   4|  0|null|
|  1|22/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   8|  1|null|
|  1|23/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  17|  3|null|
|  1|24/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  22|  3|null|
|  1|25/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  29|  6|null|
|  1|26/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  33|  6|null|
|  1|27/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  49|  9|null|
|  1|28/03/2

In [41]:
!pip install pandas

TypeError: 'DataFrameReader' object is not callable

## 1.2) lecture avec les entêtes
Recharger le même fichier mais cette fois-ci utilisez l'option header pour rajouter les noms de colonnes à votre df.

Appelez l'attribut dtypes et comparez la sortie avec celle de la lecture brute.

In [101]:
test = spark.read.format('csv').options(header=True).load(path)
test.dtypes


[('dep;date;lib_dep;lib_reg;tx_pos;tx_incid;hosp;rea;pos', 'string')]

SyntaxError: unexpected character after line continuation character (4135978901.py, line 1)

## 1.3) lecture avec les types détectés automatiquement
Recharger le fichier avec  l'option inferShema.

L'option 'inferSchema' permet de transformer les colonnes en types plus précis : entier  / booléens / chaines de caractères... bien sûr spark trouve les types uniquement si le fichier d'origine permet de les trouver de manière simple

In [102]:
test = spark.read.format('csv').options(header=True, inferSchema=True).load(path)
test.dtypes

[('dep;date;lib_dep;lib_reg;tx_pos;tx_incid;hosp;rea;pos', 'string')]

## 1.4) lecture avec schéma
Il vous permet d'afficher le schéma de votre df, avec pour chaque colonne son nom, son type, et si elle accepte les valeurs nulles ou non. 

In [103]:
test.schema

StructType(List(StructField(dep;date;lib_dep;lib_reg;tx_pos;tx_incid;hosp;rea;pos,StringType,true)))

Vous avez aussi la méthode printSchema() qui permet d'afficher le shéma du df de manière plus lisible.

In [100]:
test.printSchema()

root
 |-- dep: string (nullable = true)
 |-- date: string (nullable = true)
 |-- lib_dep: string (nullable = true)
 |-- lib_reg: string (nullable = true)
 |-- tx_pos: string (nullable = true)
 |-- tx_incid: string (nullable = true)
 |-- hosp: string (nullable = true)
 |-- rea: string (nullable = true)
 |-- pos: string (nullable = true)



# 2) écriture de la dataframe sur le disque

## 2.1) choix du format : csv

In [132]:
test.write.format("csv").save("")

IllegalArgumentException: Can not create a Path from an empty string

## 2.2) choix du format : parquet

In [119]:
test.write.format("parquet").save("./parquet")

AnalysisException: Attribute name "dep;date;lib_dep;lib_reg;tx_pos;tx_incid;hosp;rea;pos" contains invalid character(s) among " ,;{}()\n\t=". Please use alias to rename it.;

## 2.3) choix du format : json

In [134]:
df.write.save("./csv_data", format="csv")

22/11/10 09:28:40 ERROR TransportClient: Failed to send RPC RPC 5442206741629275941 to /172.19.0.7:43332: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.newClosedChannelException(AbstractChannel.java:957)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(AbstractChannel.java:865)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.write(DefaultChannelPipeline.java:1367)
	at io.netty.channel.AbstractChannelHandlerContext.invokeWrite0(AbstractChannelHandlerContext.java:717)
	at io.netty.channel.AbstractChannelHandlerContext.invokeWriteAndFlush(AbstractChannelHandlerContext.java:764)
	at io.netty.channel.AbstractChannelHandlerContext$WriteTask.run(AbstractChannelHandlerContext.java:1104)
	at io.netty.util.concurrent.AbstractEventExecutor.safeExecute(AbstractEventExecutor.java:164)
	at io.netty.util.concurrent.SingleThreadEventExecutor.runAllTasks(SingleThreadEventExecutor.java:472)
	at io.ne

In [40]:
!ls ./data_velo/Villes/

csv  parquet  ville  ville_1.csv


## 2.4) lecture de différents formats
Vous pouvez choisir de lire le df sous un format ou un autre en utilisant l'argument format dans la fonction spark.read.load

In [41]:
spark.read.load("./json_data", format="json")

DataFrame[age: bigint, casseur: boolean, home: string, id: bigint, salaire: double, sexe: string, sportif: boolean, sportivite: double, statut: string, travail: string, velo_perf_minimale: double, vitesse_a_pied: double, vitesse_a_velo: double]

In [76]:
spark.read.load(path, format="csv").count()

96355

# 3) Calculer des résultats : les actions 

## 3.1) nombre de lignes : count
Chargez les fichiers csv contenus dans le dossiers ./data_velo/Cyclistes/ dans un df nommé cyclistes, puis comptez les lignes du dataframe obtenu.

In [111]:
!ls ."hdfs://namenode:9000/Bigdata/covid.csv"

ls: cannot access '.hdfs://namenode:9000/Bigdata/covid.csv': No such file or directory


In [112]:
spark.read.load(path, format ('csv')).count()

96355

In [123]:

#df   = spark.read.load(path, format="csv")

test = spark.read.format('csv').options(header=True,sep = ";").load(path)


In [124]:
test.dtypes

[('dep', 'string'),
 ('date', 'string'),
 ('lib_dep', 'string'),
 ('lib_reg', 'string'),
 ('tx_pos', 'string'),
 ('tx_incid', 'string'),
 ('hosp', 'string'),
 ('rea', 'string'),
 ('pos', 'string')]

Afficher le schéma de ce nouveau df

In [125]:
test.printSchema()

root
 |-- dep: string (nullable = true)
 |-- date: string (nullable = true)
 |-- lib_dep: string (nullable = true)
 |-- lib_reg: string (nullable = true)
 |-- tx_pos: string (nullable = true)
 |-- tx_incid: string (nullable = true)
 |-- hosp: string (nullable = true)
 |-- rea: string (nullable = true)
 |-- pos: string (nullable = true)



Affichez 10 lignes du df.

In [126]:
test.show(10)


+---+----------+-------+--------------------+------+--------+----+---+----+
|dep|      date|lib_dep|             lib_reg|tx_pos|tx_incid|hosp|rea| pos|
+---+----------+-------+--------------------+------+--------+----+---+----+
|  1|18/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   2|  0|null|
|  1|19/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   2|  0|null|
|  1|20/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   2|  0|null|
|  1|21/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   4|  0|null|
|  1|22/03/2020|    Ain|Auvergne et Rhône...|  null|    null|   8|  1|null|
|  1|23/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  17|  3|null|
|  1|24/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  22|  3|null|
|  1|25/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  29|  6|null|
|  1|26/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  33|  6|null|
|  1|27/03/2020|    Ain|Auvergne et Rhône...|  null|    null|  49|  9|null|
+---+-------

## 3.2) moyenne : agg + colonne + mean
A l'aide de la méthode agg(), calculez la moyenne sur la colonne vitesse.

Vous pouvez récuperer le résultat avec la méthode collect().

In [220]:
test.agg({'hosp':'mean'}).show()


+------------------+
|         avg(hosp)|
+------------------+
|177.33182651991615|
+------------------+



## 3.3) quantile approximatifs pour gagner du temps de calcul
En statistiques et en théorie des probabilités, les quantiles sont les valeurs qui divisent un jeu de données en intervalles contenant le même nombre de données. Il y a donc un quantile de moins que le nombre de groupes créés. Ainsi les quartiles sont les trois quantiles qui divisent un ensemble de données en quatre groupes de taille égale.

La méthode approxQuantile permet de laisser une tolérance a l'erreur ce qui réduit le temps de calul sur d'énormes jeux de données.

In [131]:
import time
df.printSchema()

root
 |-- dep;date;lib_dep;lib_reg;tx_pos;tx_incid;hosp;rea;pos: string (nullable = true)



In [51]:
def calcul_quantile(df, erreur_acceptee):
    debut            = time.time()
    colonne          = "vitesse"
    quantiles_voulus = [0.25, 0.50, 0.75]
    resultat         =  df.approxQuantile(colonne, quantiles_voulus , erreur_acceptee )
    fin              = time.time()
    delais           = fin -debut
    print ("delais =%.2f sec, quantiles = %s"%(delais, resultat))

In [52]:
calcul_quantile(df, 0.05)

delais =5.53 sec, quantiles = [0.030000000000000006, 0.2944550644296354, 0.5931162118010243]


In [53]:
calcul_quantile(df, 0.01)

delais =4.78 sec, quantiles = [0.030000000000000006, 0.2944550644296354, 0.6295531740219638]


In [54]:
calcul_quantile(df, 0)

delais =15.12 sec, quantiles = [0.030000000000000006, 0.3283952876721612, 0.6295531740219638]


## Reload de la dataframe
Chargez le fichier villes dans un df nommé villes.

In [4]:
villes =spark.read.load("./data_velo/villes/", format="csv", header=True, inferSchema="True")
villes.printSchema()


root
 |-- id: integer (nullable = true)
 |-- vitesse_a_pied: double (nullable = true)
 |-- vitesse_a_velo: double (nullable = true)
 |-- home: string (nullable = true)
 |-- travail: string (nullable = true)
 |-- sportif: boolean (nullable = true)
 |-- casseur: boolean (nullable = true)
 |-- statut: string (nullable = true)
 |-- salaire: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sportivite: double (nullable = true)
 |-- velo_perf_minimale: double (nullable = true)



## 3.4) corrélation
En probabilités et en statistique, la corrélation entre plusieurs variables aléatoires ou statistiques est une notion de liaison qui contredit leur indépendance.

Calculez la corrélation entre les colonnes age et vitesse_a_velo.

In [56]:

villes.corr(col1= 'age', col2= 'vitesse_a_velo')

-0.06411845578664936

## 3.5) covariance
Calculez la covariance entre les colonnes age et vitesse_a_velo.

In [57]:

villes.cov ('age', 'vitesse_a_velo')

-1.5721945755314064

## 3.6) sample

In [58]:
villes_1_pct = villes.sample(False, 0.1)


In [59]:
villes_1_pct.count()

6

In [60]:
villes.exceptAll(villes_1_pct).count()

44

## 3.7) filter 
La méthode filter() permet le df selon certaines valeurs dans les colonnes.

Utilisez cette méthode pour récuperer seulement les lignes avec le sexe féminin.

In [61]:
villes.filter("sexe ='F'").show()

+---+-------------------+-------------------+-------------------+-------------------+-------+-------+--------------------+------------------+----+---+-------------------+------------------+
| id|     vitesse_a_pied|     vitesse_a_velo|               home|            travail|sportif|casseur|              statut|           salaire|sexe|age|         sportivite|velo_perf_minimale|
+---+-------------------+-------------------+-------------------+-------------------+-------+-------+--------------------+------------------+----+---+-------------------+------------------+
|  3|               0.02|               0.05|(lon:3.79 lat:3.81)|(lon:0.20 lat:0.21)|  false|  false|technicien_de_sur...| 20026.72646423192|   F| 20|                0.1|               0.4|
|  4|               0.02|               0.05|(lon:3.39 lat:0.93)|(lon:0.58 lat:0.20)|  false|  false|technicien_de_sur...|15214.584161640825|   F| 35|                0.1|               0.4|
|  6|               0.02|               0.05|(lon:

In [62]:
villes =spark.read.load("./data_velo/villes/", format="csv", header=True, inferSchema="True")
villes.printSchema()

root
 |-- id: integer (nullable = true)
 |-- vitesse_a_pied: double (nullable = true)
 |-- vitesse_a_velo: double (nullable = true)
 |-- home: string (nullable = true)
 |-- travail: string (nullable = true)
 |-- sportif: boolean (nullable = true)
 |-- casseur: boolean (nullable = true)
 |-- statut: string (nullable = true)
 |-- salaire: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sportivite: double (nullable = true)
 |-- velo_perf_minimale: double (nullable = true)



# 4) Transformer la données : les transformations!

## Transformations : demandent à être suivi par un collect ou une action (count par exemple)

## 4.1) obtenir des statistiques sur les colonnes numériques
La méthode describe() permet de calculer les statistiques récapitulatives d'une ou plusieurs colonnes numériques dans un df. Si le nom des colonnes n'est pas spécifié, la méthode calculera des statistiques récapitulatives pour toutes les colonnes numériques présentes dans le df.

Afficher les statistiques de la colonne age.

In [63]:
villes.describe(["sexe", "age"]).describe().collect()

[Row(summary='count', summary='5', sexe='3', age='5'),
 Row(summary='mean', summary=None, sexe='50.0', age='43.43979797464467'),
 Row(summary='stddev', summary=None, sexe='NaN', age='27.13153036418277'),
 Row(summary='min', summary='count', sexe='50', age='16'),
 Row(summary='max', summary='stddev', sexe='H', age='83')]

## 4.2) groupby
La méthode groupBy() suivie de la methode agg() permet de grouper le df selon les catgories d'une ou plusieurs colonnes pour faire des calculs sur ces catégories.

Calculez la moyenne de la colonnes sportivité selon le sexe des personnes.

In [64]:
villes.groupBy('sexe').agg({"sportivite":'mean'}).show()

+----+------------------+
|sexe|   avg(sportivite)|
+----+------------------+
|   F|1.8410619134680517|
|   H|1.6356186755623958|
+----+------------------+



Calculez la moyenne de la colonne age et la valeur max de la colonne sportivité par sexe.

In [65]:
villes.groupBy('sexe').agg({"age":'mean' , "sportivite" :'max'}).show()

+----+-----------------+------------------+
|sexe|  max(sportivite)|          avg(age)|
+----+-----------------+------------------+
|   F|5.304311048875953|46.095238095238095|
|   H|7.814024407120282| 50.06896551724138|
+----+-----------------+------------------+



Calculez la moyenne des colonnes vitesse_a_pied et vitesse_a_velo par sexe.

In [66]:
villes.groupBy('sexe').agg({"vitesse_a_pied":'mean' , "vitesse_a_velo" :'mean'}).show()

+----+-------------------+-------------------+
|sexe|avg(vitesse_a_velo)|avg(vitesse_a_pied)|
+----+-------------------+-------------------+
|   F| 0.9205309567340259|0.36821238269361034|
|   H| 1.3084949404499162| 0.4906856026687184|
+----+-------------------+-------------------+



## 4.3) summary 
La méthode summary() permet des faire des calculs statistiques de base sur toutes les colonnes du df.


In [67]:
villes.summary("count", "max").collect()

[Row(summary='count', id='50', vitesse_a_pied='50', vitesse_a_velo='50', home='50', travail='50', statut='50', salaire='50', sexe='50', age='50', sportivite='50', velo_perf_minimale='50'),
 Row(summary='max', id='51', vitesse_a_pied='2.344207322136085', vitesse_a_velo='6.251219525696226', home='(lon:3.90 lat:0.62)', travail='(lon:3.91 lat:3.22)', statut='éboueur', salaire='148702.7189509448', sexe='H', age='83', sportivite='7.814024407120282', velo_perf_minimale='0.4')]

## 4.4) union de dataframe

#### ajouter les colonnes les unes à côté des autres : join

In [68]:
villes.join(villes, on="id").printSchema()

root
 |-- id: integer (nullable = true)
 |-- vitesse_a_pied: double (nullable = true)
 |-- vitesse_a_velo: double (nullable = true)
 |-- home: string (nullable = true)
 |-- travail: string (nullable = true)
 |-- sportif: boolean (nullable = true)
 |-- casseur: boolean (nullable = true)
 |-- statut: string (nullable = true)
 |-- salaire: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sportivite: double (nullable = true)
 |-- velo_perf_minimale: double (nullable = true)
 |-- vitesse_a_pied: double (nullable = true)
 |-- vitesse_a_velo: double (nullable = true)
 |-- home: string (nullable = true)
 |-- travail: string (nullable = true)
 |-- sportif: boolean (nullable = true)
 |-- casseur: boolean (nullable = true)
 |-- statut: string (nullable = true)
 |-- salaire: double (nullable = true)
 |-- sexe: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- sportivite: double (nullable = true)
 |-- velo_perf_minimale: double (n

#### ajouter les lignes les unes sous les autres : union

In [69]:
villes.unionByName(villes).count()

100

## 4.5) filtre

In [70]:
villes.where(villes.sexe=="F").count()

21

## 4.6) concaténation de colonne : F.concat

In [5]:
from pyspark.sql.types import *
from pyspark.sql       import functions as F

Nous allons ici reprendre le df cyclistes.

Utiliser les méthodes withColumn() et F.concat() pour ajouter une colonne au df qui contiendra la concatenation des valeurs des colonnes id et sur_velo. 

In [6]:
path = "./data_velo/Cyclistes/*.csv" 
tous_les_cyclistes = spark.read.format("csv").option("header", "true").load(path, inferSchema=True)
tous_les_cyclistes.count()
tous_les_cyclistes.withColumn("id_sur_velo", F.concat(tous_les_cyclistes.id,  tous_les_cyclistes.sur_velo)).first()

Row(id=12, timestamp='2018-01-01 00:01:00', sur_velo=False, velo='False', vitesse=0.030000000000000006, position='(lon:2.07 lat:1.24)', destination_finale='False', id_sur_velo='12false')

# 5) udf 
Il est possible d'enregistrer des fonctions python que l'on écrit nous même pour les appliquer sur une colonne d'une dataframe, c'est ce qu'on appelle les udf, pour User Defined Functions

In [73]:
from pyspark.sql.types     import *
from pyspark.sql.functions import udf

@udf(returnType = FloatType())
def cube(colonne):
    return colonne*colonne

In [74]:
villes.select(cube("salaire")).show()

+-------------+
|cube(salaire)|
+-------------+
|1.82702162E10|
|  4.0106976E8|
| 2.31483568E8|
| 3.32549088E8|
|  9.5185338E8|
| 2.96969139E9|
| 1.12880461E9|
| 1.39238304E8|
|  1.0977216E9|
|  2.8854487E9|
|  6.4748621E8|
| 1.22958438E9|
|  5.5217725E9|
| 2.57463706E9|
|  4.4269376E8|
| 2.87179296E8|
| 4.88372224E8|
|  6.9554765E9|
|  6.1511916E9|
|   5.757398E9|
+-------------+
only showing top 20 rows



In [75]:
villes.select(cube(villes.salaire)).show()

+-------------+
|cube(salaire)|
+-------------+
|1.82702162E10|
|  4.0106976E8|
| 2.31483568E8|
| 3.32549088E8|
|  9.5185338E8|
| 2.96969139E9|
| 1.12880461E9|
| 1.39238304E8|
|  1.0977216E9|
|  2.8854487E9|
|  6.4748621E8|
| 1.22958438E9|
|  5.5217725E9|
| 2.57463706E9|
|  4.4269376E8|
| 2.87179296E8|
| 4.88372224E8|
|  6.9554765E9|
|  6.1511916E9|
|   5.757398E9|
+-------------+
only showing top 20 rows



In [76]:
villes.select(cube("salaire")).collect()
#villes.withColumn("salaire",cube("salaire")).collect()

[Row(cube(salaire)=18270216192.0),
 Row(cube(salaire)=401069760.0),
 Row(cube(salaire)=231483568.0),
 Row(cube(salaire)=332549088.0),
 Row(cube(salaire)=951853376.0),
 Row(cube(salaire)=2969691392.0),
 Row(cube(salaire)=1128804608.0),
 Row(cube(salaire)=139238304.0),
 Row(cube(salaire)=1097721600.0),
 Row(cube(salaire)=2885448704.0),
 Row(cube(salaire)=647486208.0),
 Row(cube(salaire)=1229584384.0),
 Row(cube(salaire)=5521772544.0),
 Row(cube(salaire)=2574637056.0),
 Row(cube(salaire)=442693760.0),
 Row(cube(salaire)=287179296.0),
 Row(cube(salaire)=488372224.0),
 Row(cube(salaire)=6955476480.0),
 Row(cube(salaire)=6151191552.0),
 Row(cube(salaire)=5757398016.0),
 Row(cube(salaire)=1513707392.0),
 Row(cube(salaire)=845977152.0),
 Row(cube(salaire)=15607006208.0),
 Row(cube(salaire)=279298720.0),
 Row(cube(salaire)=658956352.0),
 Row(cube(salaire)=332481376.0),
 Row(cube(salaire)=11568824320.0),
 Row(cube(salaire)=871186176.0),
 Row(cube(salaire)=731495360.0),
 Row(cube(salaire)=2211249

# 6)	Etude de cas : analyse des fichiers de logs des cyclistes

In [7]:
from pyspark.sql.types import *
from pyspark.sql       import functions as F
from pyspark.sql.types     import *
from pyspark.sql.functions import udf

## 6.1)  Charger la donnée dans un df nommé cycliste

In [8]:

cyclistes =spark.read.load("./data_velo/Cyclistes/",format = "csv", header=True, inferSchema="True")

## 6.2) vérifier le nombre de cycles

Comptez le nombre d'id uniques.

In [9]:
cyclistes.select("id").distinct().count()


50

## 6.3) transformer les timestamp en date

Voici une fonction qui permert de récuperer la date sous forme de chaîne de caractère dans la colonne timestamps pour la transformer en date exploitable en tant que telle.

Créez une nouvelle colonne dans votre df cycliste stockant le résultat de cette fonction.

In [10]:
from pyspark.sql.types import TimestampType

@udf(returnType = TimestampType())
def transform_timestamp_in_date(timestamp):
    from datetime import datetime
    return datetime.strptime(str(timestamp), "%Y-%m-%d %H:%M:%S")

In [116]:
cyclistes.withColumn("Timestamp2",transform_timestamp_in_date(cyclistes['timestamp'])).show()

+---+-------------------+--------+-----+--------------------+-------------------+------------------+-------------------+
| id|          timestamp|sur_velo| velo|             vitesse|           position|destination_finale|         Timestamp2|
+---+-------------------+--------+-----+--------------------+-------------------+------------------+-------------------+
| 12|2018-01-01 00:01:00|   false|False|0.030000000000000006|(lon:2.07 lat:1.24)|             False|2018-01-01 00:01:00|
| 12|2018-01-01 00:02:00|   false|False|0.030000000000000006|(lon:2.07 lat:1.24)|             False|2018-01-01 00:02:00|
| 12|2018-01-01 00:03:00|   false|False|0.030000000000000006|(lon:2.07 lat:1.24)|             False|2018-01-01 00:03:00|
| 12|2018-01-01 00:04:00|   false|False|0.030000000000000006|(lon:2.07 lat:1.24)|             False|2018-01-01 00:04:00|
| 12|2018-01-01 00:05:00|   false|False|0.030000000000000006|(lon:2.07 lat:1.24)|             False|2018-01-01 00:05:00|
| 12|2018-01-01 00:06:00|   fals

## vérifier les comptes

## 6.4) Durée des trajets par id.

A partir d'ici, il s'agit de traiter votre donnée pour récupérer la durée de chaque trajet effectué par chaque id.

1) trouvez les dates min/max par état de sur_velo, puis par id ET par état de sur_velo.

In [137]:
cyclistes.groupby(["id","sur_velo"]).agg(F.min(cyclistes["timestamp"]) , F.max(cyclistes["timestamp"])).sort("id").show()



+---+--------+-------------------+-------------------+
| id|sur_velo|     min(timestamp)|     max(timestamp)|
+---+--------+-------------------+-------------------+
|  2|    true|2018-01-01 12:33:00|2018-01-31 20:02:00|
|  2|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
|  3|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
|  3|    true|2018-01-06 06:24:00|2018-01-28 19:33:00|
|  4|    true|2018-01-06 09:48:00|2018-01-28 15:19:00|
|  4|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
|  5|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
|  5|    true|2018-01-06 12:35:00|2018-01-28 16:04:00|
|  6|    true|2018-01-01 07:07:00|2018-01-30 20:01:00|
|  6|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
|  7|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
|  7|    true|2018-01-01 06:33:00|2018-01-31 18:43:00|
|  8|    true|2018-01-06 05:14:00|2018-01-28 19:44:00|
|  8|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
|  9|    true|2018-01-06 06:54:00|2018-01-28 14:03:00|
|  9|   fa

In [11]:
cyclistes.groupby(["sur_velo"]).agg(F.min(cyclistes["timestamp"]) , F.max(cyclistes["timestamp"])).show()


+--------+-------------------+-------------------+
|sur_velo|     min(timestamp)|     max(timestamp)|
+--------+-------------------+-------------------+
|    true|2018-01-01 01:47:00|2018-01-31 21:32:00|
|   false|2018-01-01 00:01:00|2018-02-01 00:00:00|
+--------+-------------------+-------------------+



## 6.5) détecter les changements d'états "sur un vélo" ou pas : window et lag
2) Le résultat n'est pas trés pertinent, il faudrait plutôt le début et la fin de chaque trajet par id. Pour cela, il faudrait détecter les changements d'états "sur_vélo".
Utilisez la classe Window() et la fonction F.lag() pour créer une nouvelle colonne que vous appellerez changement, contenant un 0 si l'état précedent de sur_velo est le même et un 1 si l'état vient de changer (fonction changement() ci-dessous) pour chaque id.

In [ ]:
from pyspark.sql.functions import udf

In [12]:
@udf(returnType = IntegerType())
def changement(etat_actuel, etat_precedent):
    """
    Détecte si les deux états sont différent.
    
    Parametres :
        etat_actuel : valeur sur la ligne courante
                      renvoyée par F.lag (0)
        etat_precedent : valeur sur la ligne précédente
                      renvoyée par F.lag(1)
    Return: 0 s'ils sont égaux, 1 s'il y a une différence
    """
    if etat_precedent == None:
        return 0
    if etat_precedent == etat_actuel:
        return 0
    if etat_actuel != etat_precedent:
        return 1

In [13]:
from pyspark.sql.window import Window
w = Window.orderBy(["id", "date"])

collecter les données d'un id et verifier sa position lors des changements d'etats

SyntaxError: invalid syntax (462676745.py, line 1)

## 6.6) somme partielle par sous groupe : windows
Grâce à la fonction window appliquez la fonction somme() sur la colonne changement pour numeroter les trajets pour chaque id et stocker les résulats dans une nouvelle colonne appelée numero_de_trajet.


[windows fonctions](https://www.quantmetry.com/blog/window-functions-spark/)


In [ ]:
@udf(returnType = IntegerType())
def somme(indice_actuel, indice_precedent):
    if indice_precedent == None:
        return 0
    return indice_actuel + indice_precedent

## 6.7) Calculer la durée du trajet

Il suffit maintenant de repêter la première étape, c'est a dire récupérer la début et la fin de chaque trajet pour chaque id. Puis calculer la durée des trajets.

# 7) datavisualisation
Convertissez votre dataframe pyspark en dataframe pandas.

A l'aide de la librairie seaborn, réalisez un graphique en barre montrant la durée de tout les trajets.

Faire le même graphique mais cette fois-ci, faire en sorte qu'on puisse choisir un id et afficher seulement les trajets de cet id.

Sauvegardez votre dataset trajets au format csv dans le dossier data.